# #1

In [34]:
%pip install semantic-link

Note: you may need to restart the kernel to use updated packages.


ERROR: Ignored the following versions that require a different python version: 0.0.13 Requires-Python >=3.10; 0.0.17 Requires-Python >=3.10; 0.0.18 Requires-Python >=3.10; 0.0.19 Requires-Python >=3.10; 0.3.0 Requires-Python >=3.10; 0.3.1 Requires-Python >=3.10; 0.3.2 Requires-Python >=3.10; 0.3.4 Requires-Python >=3.10; 0.3.5 Requires-Python >=3.10
ERROR: Could not find a version that satisfies the requirement semantic-link (from versions: none)
ERROR: No matching distribution found for semantic-link


In [35]:
pip install python-power-bi

Note: you may need to restart the kernel to use updated packages.

In [36]:
import sempy.fabric as fabric

ModuleNotFoundError: No module named 'sempy'

# #2

In [1]:
pip install pyodbc pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pyodbc
import pandas as pd

In [9]:
conn = pyodbc.connect('DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=POC_Artificial_Intelligence_Sample.pbix')

InterfaceError: ('IM002', '[IM002] [Microsoft][ODBC Driver Manager] Data source name not found and no default driver specified (0) (SQLDriverConnect)')

In [ ]:
`

# #3

In [31]:
pip install PyDaxExtract

Note: you may need to restart the kernel to use updated packages.


# #4

In [10]:
import json

def extract_pbix_metadata(pbit_file_path):
  """Extracts the metadata from a PBIX file.

  Args:
    pbit_file_path: The path to the PBIT file.

  Returns:
    A dictionary containing the metadata for the PBIX file.
  """

  with open(pbit_file_path, "r") as f:
    data_model_schema = json.load(f)

  # Extract the metadata from the DataModelSchema file.
  metadata = {}
  metadata["tables"] = data_model_schema["tables"]
  metadata["measures"] = data_model_schema["measures"]
  metadata["calculated_columns"] = data_model_schema["calculatedColumns"]

  return metadata

# Example usage:

pbit_file_path = "POC_Artificial_Intelligence_Sample.pbit"
metadata = extract_pbix_metadata(pbit_file_path)

# Print the metadata.
print(metadata)


UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 152: character maps to <undefined>

# #5

In [37]:
import zipfile
import os
import json

def extract_metadata(pbix_file_path):
    # Open the .pbix file as a zip file
    with zipfile.ZipFile(pbix_file_path, 'r') as zip_ref:
        # Extract the contents to a temporary folder
        temp_folder = "temp_extract"
        zip_ref.extractall(temp_folder)

        # Print the list of files in the temporary folder
        print("Extracted files:", os.listdir(temp_folder))

        # Locate and read the metadata file
        metadata_path = f"{temp_folder}/Metadata"
        with open(metadata_path, 'r', encoding='latin-1') as metadata_file:
            # Read the content of the file
            metadata_content = metadata_file.read()

            # Remove null bytes before decoding JSON
            cleaned_content = metadata_content.replace('\x00', '')
            
            # Manually attempt to decode the JSON string
            try:
                metadata = json.loads(cleaned_content)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")
                metadata = None

    # Cleanup: Remove the temporary folder
    # Note: Comment this out if you want to inspect the extracted files
    import shutil
    shutil.rmtree(temp_folder)

    return metadata

# Example usage
pbix_file_path = 'POC_Artificial_Intelligence_Sample.pbix'
metadata = extract_metadata(pbix_file_path)

# Print the extracted metadata
if metadata is not None:
    print(json.dumps(metadata, indent=2))


Extracted files: ['Connections', 'DataModel', 'DiagramLayout', 'docProps', 'Metadata', 'Report', 'SecurityBindings', 'Settings', 'Version', '[Content_Types].xml']
{
  "Version": 5,
  "AutoCreatedRelationships": [],
  "CreatedFrom": "Cloud",
  "CreatedFromRelease": "2021.10"
}


# #6

In [44]:
import json
from zipfile import ZipFile
import shutil
import pandas as pd

class ReportExtractor():

    def __init__(self, path, name):
        self.path = path
        self.name = name
        self.result = []

    def extract(self):
        pathFolder = f'{self.path}/temp_{self.name[:-5]}'
        try: shutil.rmtree(pathFolder)
        except:
            print(f'folder {pathFolder} not present')
        f = ZipFile(f'{self.path}/{self.name}', 'r')
        f.extractall(pathFolder)
        report_layout = json.loads(
            open(f'{pathFolder}/Report/Layout', 'r', encoding='utf-16 le').read()
        )

        f.close()
        fields = []
        for s in report_layout['sections']: 
            for vc in s['visualContainers']:       
                try:
                    query_dict = json.loads(vc['config'])
                    for command in query_dict['singleVisual']['prototypeQuery']['Select']:

                        if 'Measure' in command.keys():
                            #- MEASURES
                            name = command['Name'].split('.')[1]
                            table = command['Name'].split('.')[0]
                            fields.append([s['displayName'], query_dict['name'], table, name, 'Measure'])

                        elif 'Column' in command.keys():
                            # COLUMNS
                            name = command['Name'].split('.')[1]
                            table = command['Name'].split('.')[0]
                            fields.append([s['displayName'], query_dict['name'], table, name, 'Column'])

                        elif 'Aggregation' in command.keys():
                            # AGGREGATIONS
                            if ( ( '(' in command['Name'] ) & ( ')' in command['Name'] ) ): 
                                txt_extraction = command['Name'][command['Name'].find('(') + 1: command['Name'].find(')') ]
                            name  = txt_extraction.split('.')[1]
                            table = txt_extraction.split('.')[0]
                            fields.append([s['displayName'], query_dict['name'], table, name, 'Aggregation'])

                except: 
                    pass
        self.result =  fields
        shutil.rmtree(pathFolder)
        
        
re = ReportExtractor("C:/Users/SRNADIMP/OneDrive - Capgemini/New Laptop/OneDrive - Capgemini/Desktop/BI_Report_Template_Matching/", "POC_Artificial_Intelligence_Sample.pbix")
re.extract()

df = pd.DataFrame(re.result, columns=['Page', 'Visual ID', 'Table', 'Name', 'Type'])
print(df.head)

folder C:/Users/SRNADIMP/OneDrive - Capgemini/New Laptop/OneDrive - Capgemini/Desktop/BI_Report_Template_Matching//temp_POC_Artificial_Intelligence_Sample not present
<bound method NDFrame.head of                   Page             Visual ID                 Table  \
0      Key Influencers  01af2f16cdd3b0443588         Opportunities   
1      Key Influencers  01af2f16cdd3b0443588              Products   
2      Key Influencers  ec80685720004c638d61         Opportunities   
3      Key Influencers  ec80685720004c638d61                Owners   
4      Key Influencers  ec80685720004c638d61         Opportunities   
5      Key Influencers  ec80685720004c638d61              Products   
6      Key Influencers  ec80685720004c638d61         Opportunities   
7      Key Influencers  ec80685720004c638d61                Owners   
8      Key Influencers  ec80685720004c638d61              Accounts   
9      Key Influencers  ec80685720004c638d61         Opportunities   
10     Key Influencers  ec8068572

In [39]:
"""1) Chat Bot

    a. Extract the tables and fields associated with the reports (Meta data)
    b. Extract the visual header and x,y axis
"""

'1) Chat Bot\n\n    a. Extract the tables and fields associated with the reports (Meta data)\n    b. Extract the visual header and x,y axis\n'